This is a retrospective data analysis of our ESPN Fantasy Football league data at the end of Week 17 for the 2022 season. The code can be re-used for any past season using the same ESPN API.

I used the [Pro Football Reference](https://www.pro-football-reference.com/years/2022/games.htm) for NFL schedules and [Christian Wendt's ESPN Fantasy Football API](https://github.com/cwendt94/espn-api) to extract ESPN league-specific data.

For data visualization, I used the [Altair Data Visualization Library](https://altair-viz.github.io/). Altair was chosen for its interactivity, especially for its ability to display hover data and the compatibility of interactive plots with GitHub Pages.

One question this analysis can answer is: what was the return on investment of a given players with fantasy draft or waiver budget dollars (FAAB).

See src directory for source code.

In [88]:
%cd /Users/jonathancheng/PycharmProjects/espnff/src/
import streamlit as st
import nfl_schedule as nf
import ff_league_data as ff
import calc_best_waiver as cbw
from plotting import scatterplot_acquisitions


/Users/jonathancheng/PycharmProjects/espnff/src


## Get NFL Schedule

In [2]:
year_of_interest=2022

path = r'/Users/jonathancheng/PycharmProjects/espnff/data'

league_id = 1094090
year = year_of_interest

swid = "{F191FB8C-DB2D-4D24-91FB-8CDB2DED249D}"
s2='AECJMQHsUHB0FTXdZkw93uY7GRbX8BPnm93Ye6AwvwrMsrZFGg1Lbmi07SWVov2ioN8zGMFDzZiiDSeQCa7WQHaGivGnMfGWLjmfGwkOeLXb5baD1sltp%2B%2BIfHAtl98TpmHgB16ZpGn6g3Bm5vLEA7yDC6HkbD3LSp0E2rGB7hKziLMvZ7mT6ONJFRe8Xp3ApYWSvxPr9cz0pJiI%2FF0blsZ8hyATDJMEyaQ2O%2FypcsViORr6hqYTmXHPuPKnMBfvYC8LQqi1exGw3vnyg6ptsB2Y'

espn_s2 = s2


In [3]:
df_proteam_schedule = nf.get_nfl_schedule(year_of_interest)
season_start_date = nf.get_season_start_date(df_proteam_schedule)

## Generate League object

In [4]:
league = ff.fetch_espn_api(league_id, year, espn_s2, swid)
activity_ls = league.recent_activity(1000000)
wk_ls = ff.get_weeks(league)

## Get Acquisitions Data

In [6]:
# fetch league data, wrangle into acquisitions DataFrame

acq_data_flat_ls = ff.get_acq_ls(activity_ls)

df_acq = ff.build_df_acq(acq_data_flat_ls)

## Get Draft Data

In [7]:
df_draft,drafted_players = ff.build_df_draft(league)

## Get total points of rostered players Dataframe

In [8]:
df_rostered = ff.build_df_rostered(league)

## Get total points of free agent players Dataframe

In [9]:
df_FA = ff.build_df_FA(league)

In [10]:
# Generate all player stats dataframe, including all Free Agents
df_player_stats = ff.build_df_player_stats(df_rostered,df_FA)

In [11]:
df_draft_stats = ff.build_df_draft_stats(df_draft,df_player_stats)
df_acq_stats = ff.build_df_acq_stats(df_acq,df_player_stats)
df_acq_final = ff.build_df_acq_final(season_start_date, df_draft_stats, df_acq_stats, drafted_players)

## Get player_box_scores from fantasy season

In [13]:
df_player_box_scores = ff.build_df_player_box_scores(league, wk_ls)

## Construct df_stints

In [15]:
df_stints=ff.build_df_stints(df_acq_final, df_proteam_schedule, df_player_stats, drafted_players, df_player_box_scores)

In [21]:

df_stints['Total points per stint'] = df_stints.apply(lambda x: ff.get_total_pts_per_player(x['Player'], x['Stint (wks)'], df_player_box_scores),axis=1).fillna(0)

In [25]:
df_stints[df_stints['Player'].str.contains('Koo')]

,Stint_id,Player,Team,ProTeam,Added,Bid Amount ($),Dropped,Stint (wks),Position,Drafted,Total points per stint
401,401,Younghoe Koo,Door City,ATL,2022-09-07,1,2022-12-21,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15]",K,True,115.0
402,402,Younghoe Koo,Door City,ATL,2022-12-29,1,NaT,[17],K,True,8.0


In [41]:
def merge_lists(series):
    return list(set([item for sublist in series for item in sublist]))
    # return list(itertools.chain(*ls))

In [42]:
df_temp=df_stints.groupby(by=['Player', 'Team']).agg({'Total points per stint':'sum', 'Stint (wks)':list}).reset_index()

In [45]:
df_temp['Stint (wks)'] = df_temp['Stint (wks)'].apply(lambda x: list(itertools.chain(*x)))

In [46]:
df_temp

,Player,Team,Total points per stint,Stint (wks)
0,49ers D/ST,Big Joshy Style,140.0,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1..."
1,A.J. Brown,Ice City USA,244.1,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1..."
2,AJ Dillon,The Genaissance,150.3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15..."
3,Aaron Jones,Kirk-life Balance,212.8,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15..."
4,Aaron Rodgers,Frankel's Cankles,226.6,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15..."
...,...,...,...,...
368,Younghoe Koo,Door City,123.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15..."
369,Zach Ertz,Frankel's Cankles,92.1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
370,Zamir White,Sweetless in Seattle,4.7,"[12, 13, 14, 15, 16]"
371,Zay Jones,Kittle Me Elmo,57.7,"[13, 14, 15, 16, 17]"


In [51]:
df_stints[['Player','Team','ProTeam','Position','Drafted']].drop_duplicates()

,Player,Team,ProTeam,Position,Drafted
0,49ers D/ST,Big Joshy Style,SF,D/ST,True
1,A.J. Brown,Ice City USA,PHI,WR,True
2,AJ Dillon,The Genaissance,GB,RB,True
3,Aaron Jones,Kirk-life Balance,GB,RB,True
4,Aaron Rodgers,Frankel's Cankles,GB,QB,True
...,...,...,...,...,...
401,Younghoe Koo,Door City,ATL,K,True
403,Zach Ertz,Frankel's Cankles,ARI,TE,True
404,Zamir White,Sweetless in Seattle,OAK,RB,False
405,Zay Jones,Kittle Me Elmo,JAX,WR,False


In [53]:
df_player_ffteam = df_temp.merge(df_stints[['Player','Team','ProTeam','Position','Drafted']].drop_duplicates(), how='left')

In [60]:
for idx, x in df_player_ffteam.iterrows():
    if x['Stint (wks)']:
        df_player_ffteam.loc[idx,'quantile'] = cbw.calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'],df_player_box_scores)
    else:
        df_player_ffteam.loc[idx,'quantile'] = 0
        

In [62]:
df_player_ffteam['Num weeks'] = df_player_ffteam['Stint (wks)'].apply(lambda x: len(x))

In [91]:
df_player_ffteam

,Player,Team,Total points per stint,Stint (wks),ProTeam,Position,Drafted,quantile,Num weeks
0,49ers D/ST,Big Joshy Style,140.0,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",SF,D/ST,True,96.551724,16
1,A.J. Brown,Ice City USA,244.1,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",PHI,WR,True,96.052632,16
2,AJ Dillon,The Genaissance,150.3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",GB,RB,True,69.863014,16
3,Aaron Jones,Kirk-life Balance,212.8,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",GB,RB,True,91.780822,16
4,Aaron Rodgers,Frankel's Cankles,226.6,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",GB,QB,True,75.000000,16
...,...,...,...,...,...,...,...,...,...
368,Younghoe Koo,Door City,123.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",ATL,K,True,88.636364,15
369,Zach Ertz,Frankel's Cankles,92.1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",ARI,TE,True,92.592593,10
370,Zamir White,Sweetless in Seattle,4.7,"[12, 13, 14, 15, 16]",OAK,RB,False,14.285714,5
371,Zay Jones,Kittle Me Elmo,57.7,"[13, 14, 15, 16, 17]",JAX,WR,False,75.000000,5


In [75]:
df_waiver = df_player_ffteam[(~df_player_ffteam['Drafted']) & (df_player_ffteam['Num weeks']>=8)]

In [76]:
df_waiver.sort_values('quantile',ascending=False)

,Player,Team,Total points per stint,Stint (wks),ProTeam,Position,Drafted,quantile,Num weeks
32,Brett Maher,Ice City USA,103.0,"[8, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17]",DAL,K,False,100.000000,11
139,Eagles D/ST,Ice City USA,135.0,"[3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16,...",PHI,D/ST,False,100.000000,14
202,Jason Myers,is the fantasy season everett?,74.0,"[8, 9, 10, 12, 13, 14, 15, 16, 17]",SEA,K,False,100.000000,9
226,Justin Fields,Kittle Me Elmo,172.9,"[9, 10, 11, 12, 13, 15, 16, 17]",CHI,QB,False,96.153846,8
193,Jamaal Williams,Frankel's Cankles,200.7,"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 1...",DET,RB,False,87.837838,16
344,Trevor Lawrence,is the fantasy season everett?,211.4,"[6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17]",JAX,QB,False,85.714286,11
331,Taysom Hill,is the fantasy season everett?,79.2,"[6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17]",NO,TE,False,84.615385,11
210,Jets D/ST,Kittle Me Elmo,44.0,"[7, 11, 12, 13, 14, 15, 16, 17]",NYJ,D/ST,False,82.142857,8
85,D'Onta Foreman,Kittle Me Elmo,102.5,"[8, 9, 10, 11, 12, 14, 15, 16, 17]",CAR,RB,False,75.757576,9
157,Geno Smith,The Genaissance,191.6,"[6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17]",SEA,QB,False,75.000000,11


In [78]:
df_stints.head()

,Stint_id,Player,Team,ProTeam,Added,Bid Amount ($),Dropped,Stint (wks),Position,Drafted,Total points per stint
0,0,49ers D/ST,Big Joshy Style,SF,2022-09-07,1,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",D/ST,True,140.0
1,1,A.J. Brown,Ice City USA,PHI,2022-09-07,34,NaT,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",WR,True,244.1
2,2,AJ Dillon,The Genaissance,GB,2022-09-07,4,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,150.3
3,3,Aaron Jones,Kirk-life Balance,GB,2022-09-07,42,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,212.8
4,4,Aaron Rodgers,Frankel's Cankles,GB,2022-09-07,3,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",QB,True,226.6


In [79]:
for idx, x in df_stints.iterrows():
    if x['Stint (wks)']:
        df_stints.loc[idx,'quantile'] = cbw.calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'],df_player_box_scores)
    else:
        df_stints.loc[idx,'quantile'] = 0
        

In [80]:
df_stints

,Stint_id,Player,Team,ProTeam,Added,Bid Amount ($),Dropped,Stint (wks),Position,Drafted,Total points per stint,quantile
0,0,49ers D/ST,Big Joshy Style,SF,2022-09-07,1,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",D/ST,True,140.0,96.551724
1,1,A.J. Brown,Ice City USA,PHI,2022-09-07,34,NaT,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",WR,True,244.1,96.052632
2,2,AJ Dillon,The Genaissance,GB,2022-09-07,4,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,150.3,69.863014
3,3,Aaron Jones,Kirk-life Balance,GB,2022-09-07,42,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,212.8,91.780822
4,4,Aaron Rodgers,Frankel's Cankles,GB,2022-09-07,3,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",QB,True,226.6,75.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
402,402,Younghoe Koo,Door City,ATL,2022-12-29,1,NaT,[17],K,True,8.0,54.166667
403,403,Zach Ertz,Frankel's Cankles,ARI,2022-09-07,2,2022-11-17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",TE,True,92.1,92.592593
404,404,Zamir White,Sweetless in Seattle,OAK,2022-11-27,1,2022-12-28,"[12, 13, 14, 15, 16]",RB,False,4.7,14.285714
405,405,Zay Jones,Kittle Me Elmo,JAX,2022-11-30,5,NaT,"[13, 14, 15, 16, 17]",WR,False,57.7,75.000000


In [83]:
plot_title, chart=scatterplot_acquisitions(df_stints, select_acq_method=[True], select_positions=['QB','RB', 'WR', 'TE'])
chart.display()

alt.Chart(...)

In [84]:
import altair as alt
import pandas as pd

select_acq_method=[True] 
select_positions=['QB','RB', 'WR', 'TE']

if select_acq_method is None:
    select_acq_method = [True]

if select_positions is None:
    select_positions = ['RB', 'WR', 'TE']

g = df_stints.groupby(by=["Drafted", "Position"])

df = pd.concat([g.get_group((acq_by_draft, position))
                for acq_by_draft in select_acq_method
                for position in select_positions], axis=0)

if select_acq_method[0]:
    status = "Draft"
else:
    status = "Waiver"

positions = ', '.join(select_positions)

plot_title = f"Position: {positions} , Acquired by: {status}"
selection = alt.selection_multi(fields=["Team"], bind="legend")

color = alt.condition(
    selection,
    alt.Color(
        "Team:N",
        scale=alt.Scale(scheme="tableau20"),
    ),
    alt.value("lightgray"),
)

chart = (
    alt.Chart(df)
    .mark_circle(size=40)
    .encode(
        alt.X("Bid Amount ($)", axis=alt.Axis(grid=False)),
        alt.Y("quantile", axis=alt.Axis(grid=False)),
        color=color,
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
        tooltip=["Player", "Team", "Bid Amount ($)", "Total points per stint"],
    )
    .add_selection(selection)
    .properties(width=450, height=450, title=plot_title)
    .configure_axis(labelFontSize=18, titleFontSize=18)
    .configure_title(fontSize=20)
    .configure_legend(labelFontSize=14, titleFontSize=14)
)

In [85]:
chart.display()

alt.Chart(...)

In [36]:
import itertools
list(itertools.chain(*ls))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 17]

In [ ]:
# Build a scoring algorithm 
# Input: Position, Stint (wks), aTotal points, num weeks played
# Calculate that player's percentile against the rest of his position for the total points scored in the given weeks
# Get 1D array of the particular position within the timespan

In [ ]:
def get_total_pts_per_player(player, stint, df_player_box_scores):
    if stint:
        g = df_player_box_scores.groupby(by="Player")
        df = g.get_group(player)
        return df[df["Week"].isin(stint)]["Total points"].sum()

In [ ]:
# get quantile per stint



In [ ]:
df_stints[df_stints['Player'].str.contains('Daniel Jones')]

In [ ]:
df_stints.head()

In [ ]:
stint = [13, 14, 15, 16, 17, 18]
position = 'QB'
total_points_oneplayer = 102.9

df_temp=df_player_box_scores[(df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]
total_position_stint = df_temp.groupby(by=['Player']).agg({'Total points':'sum'}).reset_index()['Total points']
quantile = total_position_stint.quantile((total_points_oneplayer-total_position_stint.min())/(total_position_stint.max()-total_position_stint.min()))


In [ ]:
from scipy import stats

def calculate_scoring_quantile_per_stint(stint,position,total_points_oneplayer):
    
    df_allplayers_stint = df_player_box_scores[(df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]
    total_position_stint = df_allplayers_stint.groupby(by=['Player']).agg({'Total points':'sum'}).reset_index()['Total points']
    quantile = stats.percentileofscore(total_position_stint.values, total_points_oneplayer)
    return quantile


In [ ]:
df_stints=df_stints.drop(['quantile'],axis=1)

In [ ]:
for idx, x in df_stints.iterrows():
    if x['Stint (wks)']:
        df_stints.loc[idx,'quantile'] = calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'])
    else:
        df_stints.loc[idx,'quantile'] = 0
    

In [ ]:
df_stints['Num wks'] = df_stints['Stint (wks)'].apply(lambda x: len(x))

In [ ]:
df_stints1 = df_stints[df_stints['Num wks']>=7]

In [ ]:
df_stints1[~df_stints1['Drafted']].sort_values(by=['quantile'],ascending=False)

In [ ]:
for idx,x in df_stints.iterrows():
    if idx == 1:
        break

In [ ]:
x

In [ ]:
stint = x['Stint (wks)']
position = x['Position']
total_points_oneplayer = x['Total points per stint']
print(total_points_oneplayer)

df_allplayers_stint = df_player_box_scores[(df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]


In [ ]:
total_position_stint = df_allplayers_stint.groupby(by=['Player']).agg({'Total points':'sum'}).reset_index()['Total points']


In [ ]:
total_position_stint.max()

In [ ]:
total_position_stint.min()

In [ ]:
quantile = stats.percentileofscore(total_position_stint.values, total_points_oneplayer)


In [ ]:
quantile

In [ ]:
# quantile = total_position_stint.quantile((total_points_oneplayer-total_position_stint.min())/(total_position_stint.max()-total_position_stint.min()))

In [ ]:
quantile

In [ ]:
x

In [ ]:
df_stints[df_stints['Drafted']==False].sort_values(by=['quantile'],ascending=False)

In [ ]:
# df_stints1 = df_stints[df_stints['Player'].str.contains('Daniel Jones')].iloc[-1,:]
# df_stints1.apply(lambda x: calculate_scoring_quantile_per_stint(x['Stint (wks)'],x['Position'],x['Total points per stint']),axis=1)
df_stints1.apply(lambda x: calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint']), axis=1)


In [ ]:
a = df_stints[(df_stints['Player'].str.contains('Daniel Jones'))].iloc[-1,:]['Total points per stint']

In [ ]:
a

In [ ]:
stint

In [ ]:
18 in stint

In [ ]:
import pandas as pd

df_temp = pd.concat([df_temp.groupby(by='Week').get_group(i) for i in stint if i!=18])
df_temp['Total points'].sum()/df_temp.shape[0]

In [ ]:
s=df_temp['Total points']

In [ ]:
df_temp.groupby(by=['Player']).get_group('Daniel Jones')

In [ ]:
df_temp.groupby(by=['Player']).get_group('Tom Brady')['Total points'].sum()

In [ ]:
df_temp['Total points']

In [ ]:
df_temp=df_temp.groupby(by=['Player']).agg(sum).reset_index().sort_values(by='Total points', ascending=False)

In [ ]:
s = df_temp['Total points']


In [ ]:
quantile = s.quantile((a-s.min())/(s.max()-s.min()))


In [ ]:
df_temp

In [ ]:
quantile

In [ ]:
player = 'Daniel Jones'
g = df_player_box_scores.groupby(by="Player")
df = g.get_group(player)


In [ ]:
df

In [ ]:


def scatterplot_acquisitions(df_stints, select_acq_method=None, select_positions=None):
    
    if select_acq_method is None:
        select_acq_method = [True]

    if select_positions is None:
        select_positions=['RB','WR','TE']

    g = df_stints.groupby(by=["Drafted", "Position"])

    df = pd.concat([g.get_group((acq_by_draft,position)) 
           for acq_by_draft in select_acq_method
           for position in select_positions],axis=0)

    if select_acq_method[0]:
        status = "Draft"
    else:
        status = "Waiver"

    positions = ', '.join(select_positions)

    plot_title = f"Position: {positions} , Acquired by: {status}"
    selection = alt.selection_multi(fields=["Team"], bind="legend")

    color = alt.condition(
        selection,
        alt.Color(
            "Team:N",
            scale=alt.Scale(scheme="tableau20"),
        ),
        alt.value("lightgray"),
    )

    chart = (
        alt.Chart(df)
        .mark_circle(size=40)
        .encode(
            alt.X("Bid Amount ($)", axis=alt.Axis(grid=False)),
            alt.Y("Total points per stint", axis=alt.Axis(grid=False)),
            color=color,
            opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
            tooltip=["Player", "Team", "Bid Amount ($)", "Total points per stint"],
        )
        .add_selection(selection)
        .properties(width=450, height=450, title=plot_title)
        .configure_axis(labelFontSize=18, titleFontSize=18)
        .configure_title(fontSize=20)
        .configure_legend(labelFontSize=14, titleFontSize=14)
    )
    return plot_title, chart


## Draft Scatterplot

In [ ]:
plot_title, chart = scatterplot_acquisitions(df_stints, select_acq_method=[True], select_positions=['RB','WR','TE'])
chart.display()

## Waiver Scatterplot

- In waiver spending, Flex Player All Stars really spent a fortune and didn't get great return on Khalil Herbert. 
- Jamaal Williams, Jerick McKinnon, Curtis Samuel and D'Onta Foreman, were big value adds

In [ ]:
plot_title, chart = scatterplot_acquisitions(df_stints, select_acq_method=[False], select_positions=['RB','WR','TE'])
chart.display()

Curtis Samuel was the most valuable waiver wire receiver.

In [107]:
for acq_by_draft in [True,False]:
    for position in df_stints['Position'].unique():
        plot_title,chart = scatterplot_acquisitions(df_stints, select_acq_method=[acq_by_draft], select_positions=[position])
        chart.display()


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [99]:
df_player_ffteam['Position'].unique()

array(['D/ST', 'WR', 'RB', 'QB', 'TE', 'K'], dtype=object)

In [101]:
df_player_ffteam[(df_player_ffteam['Num weeks']>7) & (df_player_ffteam['Position'].isin(['QB','RB','WR','TE'])) & (~df_player_ffteam['Drafted'])].sort_values(by='quantile',ascending=False)

,Player,Team,Total points per stint,Stint (wks),ProTeam,Position,Drafted,quantile,Num weeks
226,Justin Fields,Kittle Me Elmo,172.9,"[9, 10, 11, 12, 13, 15, 16, 17]",CHI,QB,False,96.153846,8
193,Jamaal Williams,Frankel's Cankles,200.7,"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 1...",DET,RB,False,87.837838,16
344,Trevor Lawrence,is the fantasy season everett?,211.4,"[6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17]",JAX,QB,False,85.714286,11
331,Taysom Hill,is the fantasy season everett?,79.2,"[6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17]",NO,TE,False,84.615385,11
85,D'Onta Foreman,Kittle Me Elmo,102.5,"[8, 9, 10, 11, 12, 14, 15, 16, 17]",CAR,RB,False,75.757576,9
157,Geno Smith,The Genaissance,191.6,"[6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17]",SEA,QB,False,75.000000,11
348,Tua Tagovailoa,is the fantasy season everett?,166.3,"[3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15]",MIA,QB,False,74.074074,12
220,Joshua Palmer,Flex Player All Stars,82.0,"[9, 10, 11, 12, 13, 14, 15, 16]",LAC,WR,False,72.131148,8
206,Jeff Wilson Jr.,Ice City USA,135.1,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 1...",MIA,RB,False,70.270270,15
110,David Njoku,Ice City USA,61.4,"[4, 5, 6, 7, 8, 13, 14, 15, 16, 17]",CLE,TE,False,68.000000,10


In [106]:
df_player_ffteam[(df_player_ffteam['Num weeks']>7) & (df_player_ffteam['Position'].isin(['QB','RB','WR','TE'])) &(df_player_ffteam['quantile']>=90) ].sort_values(by='quantile',ascending=False)

,Player,Team,Total points per stint,Stint (wks),ProTeam,Position,Drafted,quantile,Num weeks
219,Josh Jacobs,Sweetless in Seattle,299.8,"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 1...",OAK,RB,True,100.000000,16
218,Josh Allen,Big Joshy Style,376.4,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",BUF,QB,True,100.000000,15
342,Travis Kelce,More than a Thielen,254.5,"[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 1...",KC,TE,True,100.000000,16
65,Christian McCaffrey,Frankel's Cankles,298.3,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",SF,RB,True,100.000000,16
228,Justin Jefferson,Sweetless in Seattle,302.7,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",MIN,WR,True,100.000000,16
362,Tyreek Hill,Big Joshy Style,284.4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...",MIA,WR,True,98.684211,16
108,Davante Adams,Fumble .,275.7,"[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 1...",OAK,WR,True,98.684211,16
16,Austin Ekeler,More than a Thielen,312.2,"[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 1...",LAC,RB,True,98.648649,16
330,T.J. Hockenson,Flex Player All Stars,163.5,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",MIN,TE,True,96.666667,16
291,Patrick Mahomes,Flex Player All Stars,404.6,"[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 1...",KC,QB,True,96.551724,16
